In [1]:
# 변수의 shape, type, value 확인
def p(var,_name="") :
    if _name != "" : print(f'<<{_name}>>')
    if type(var)!=type([]):
        try:
            print(f'Shape:{var.shape}')
        except :
            pass
    print(f'Type: {type(var)}')
    print(f'Values: {var}')

def pst(_x,_name=""):
    print(f'<<{_name}>> Shape{_x.shape}, {type(_x)}')
def ps(_x,_name=""):
    print(f'<<{_name}>> Shape{_x.shape}')

# LangChain RAN(Retrieval-Augmented Generation) Agent 구현

In [ ]:
%%time
!pip install -q langchain langchain-openai pydantic faiss-cpu sentence-transformers hnswlib

In [3]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [4]:
import os
from google.colab import userdata

# OpenAI API 키는 필요시 설정 (이번 실습에서는 사용하지 않음)
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## AG용 샘플 매뉴얼 파일 생성

In [5]:
%%writefile semiconductor_manual.txt
# 반도체 공정 기술 매뉴얼 v3.0

## 1. 서론
이 문서는 최신 반도체 제조 공정에 대한 기술적 세부사항을 다룹니다. 각 공정 단계는 수율과 성능에 직접적인 영향을 미치므로, 모든 파라미터는 엄격하게 관리되어야 합니다.

## 2. 증착 공정 (Deposition)
증착은 웨이퍼 위에 얇은 막(Thin Film)을 형성하는 과정입니다. 화학 기상 증착(CVD)과 물리 기상 증착(PVD)으로 나뉩니다. CVD는 가스의 화학 반응을 이용하며, PVD는 플라즈마를 이용하여 물리적으로 박막을 증착시킵니다.

## 3. 포토리소그래피 (Photolithography)
포토리소그래피, 또는 노광 공정은 웨이퍼에 회로 패턴을 새기는 핵심 단계입니다. 감광액(PR)이 도포된 웨이퍼에 마스크를 통과한 빛을 쬐어 패턴을 형성합니다. 중요 파라미터는 빛의 파장, 노광량(Dose), 그리고 초점(Focus)입니다. 파장이 짧을수록 더 미세한 회로를 만들 수 있습니다.

## 4. 식각 공정 (Etching)
식각은 불필요한 부분을 선택적으로 제거하여 회로 패턴을 완성하는 과정입니다. 습식 식각(Wet Etching)과 건식 식각(Dry Etching)이 있습니다. 건식 식각은 플라즈마를 사용하여 미세하고 수직적인 패턴을 만드는 데 유리하여 현재 주류로 사용됩니다.

## 5. 화학적 기계적 연마 (CMP)
CMP(Chemical Mechanical Polishing)는 웨이퍼 표면을 화학적, 기계적 방법을 통해 거울처럼 평탄하게 만드는 공정입니다. 다음 공정의 안정성을 위해 표면의 단차를 제거하는 것이 매우 중요하며, 슬러리의 종류와 연마 압력이 주요 변수입니다.



Writing semiconductor_manual.txt


## 임베딩 모델 로드 및 문서 분할

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# 사전 훈련된 임베딩 모델 로드
# 'all-MiniLM-L6-v2'는 384차원의 벡터를 생성하며, 영어/다국어 환경에서 좋은 성능을 보입니다.
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("임베딩 모델 로드 완료.")

# 문서 로드 및 청크(chunk) 분할
# 여기서는 간단하게 문단 단위("\n\n")로 분할합니다.
with open('semiconductor_manual.txt','r',encoding='utf-8') as f:
    docs = f.read().split("\n\n")

print(f"문서가 총 {len(docs)}개의 청크로 분할되었습니다.")
print("--- 첫 번째 청크 내용 ---")
print(docs[0])

## 문서 임베딩 및 FAISS 인덱스 생성/저장

In [7]:
import faiss

# 모든 문서 청크를 인코딩하여 벡터로 변환
print("문서 임베딩 생성 중...")
embeddings = embed_model.encode(docs, convert_to_numpy=True)
print(f"임베딩 생성 완료. 임베딩 행렬 크기: {embeddings.shape}") # (문서 청크 수, 임베딩 차원)

# FAISS 인덱스 생성
# IndexFlatL2는 L2 거리(유클리드 거리)를 사용하여 모든 벡터와 직접 비교하는 가장 기본적인 인덱스입니다.
index_faiss = faiss.IndexFlatL2(embeddings.shape[1])

# 인덱스에 임베딩 데이터 추가
index_faiss.add(embeddings)
print(f"FAISS 인덱스에 {index_faiss.ntotal}개의 벡터가 추가되었습니다.")

# 인덱스 파일 저장 (재사용을 위해)
faiss.write_index(index_faiss, 'manual_faiss.index')
print("'manual_faiss.index' 파일이 저장되었습니다.")

문서 임베딩 생성 중...
임베딩 생성 완료. 임베딩 행렬 크기: (7, 384)
FAISS 인덱스에 7개의 벡터가 추가되었습니다.
'manual_faiss.index' 파일이 저장되었습니다.


# Retrieval 함수 및 RAG 파이프라인 통합
## 사용자의 질문을 받아 관련 문서를 검색하고, 이 정보를 바탕으로 LLM이 답변을 생성하는 RAG 파이프라인을 구축합니다.


## FAISS 기반 검색(Retrieval) 함수
>질문을 벡터로 변환한 뒤, FAISS 인덱스에서 가장 유사한 문서 청크를 찾아내는 함수

In [8]:
# FAISS 인덱스에서 쿼리와 가장 관련 높은 k개의 문서를 검색
def retrieve_from_faiss(query: str, k: int = 2) -> list[str]:
    # 저장된 인덱스 로드
    index = faiss.read_index('manual_faiss.index')

    # 쿼리를 벡터로 변환
    query_vector = embed_model.encode([query], convert_to_numpy=True)

    # 인덱스 검색 (D: 거리, I: 인덱스 번호)
    distances, indices = index.search(query_vector, k)

    # 검색된 인덱스 번호에 해당하는 원본 문서 청크 반환
    return [docs[i] for i in indices[0]]

In [9]:
# --- 함수 테스트 ---
test_query = "포토리소그래피 공정의 중요 파라미터는 뭐야?"
retrieved_docs = retrieve_from_faiss(test_query, k=2)

print(f"[질문]: {test_query}")
print("\n[검색된 문서]:")
for i, doc in enumerate(retrieved_docs):
    print(f"--- 문서 {i+1} ---")
    print(doc)

[질문]: 포토리소그래피 공정의 중요 파라미터는 뭐야?

[검색된 문서]:
--- 문서 1 ---
## 3. 포토리소그래피 (Photolithography)
포토리소그래피, 또는 노광 공정은 웨이퍼에 회로 패턴을 새기는 핵심 단계입니다. 감광액(PR)이 도포된 웨이퍼에 마스크를 통과한 빛을 쬐어 패턴을 형성합니다. 중요 파라미터는 빛의 파장, 노광량(Dose), 그리고 초점(Focus)입니다. 파장이 짧을수록 더 미세한 회로를 만들 수 있습니다.
--- 문서 2 ---
## 1. 서론
이 문서는 최신 반도체 제조 공정에 대한 기술적 세부사항을 다룹니다. 각 공정 단계는 수율과 성능에 직접적인 영향을 미치므로, 모든 파라미터는 엄격하게 관리되어야 합니다.


## RAG 파이프라인 구현 및 실행
>검색 -> 보강 -> 생성의 3단계 파이프라인을 구현하고 실행



In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# LLM 초기화 (Generator 역할)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 프롬프트 템플릿 정의
# LLM에게 검색된 컨텍스트를 기반으로 답변하도록 명확하게 지시합니다.
template = """당신은 반도체 공정 전문가입니다. 주어진 컨텍스트 정보를 바탕으로 사용자의 질문에 답변해주세요.
컨텍스트에 질문과 관련없는 내용이 있다면, 컨텍스트를 무시하고 아는 대로 답변하세요.

컨텍스트:
{context}

질문:
{question}

답변:
"""
prompt = ChatPromptTemplate.from_template(template)

# RAG 체인(파이프라인) 정의
# LCEL(LangChain Expression Language)을 사용하여 파이프라인을 구성
rag_chain = (
    {"context": (lambda x: "\n---\n".join(retrieve_from_faiss(x["question"]))),
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
# --- 파이프라인 실행 ---
user_question = "포토리소그래피 공정이란 무엇이고, 왜 중요한가요?"
print(f"[사용자 질문]: {user_question}\n")

print("[RAG 파이프라인 답변]:")
# invoke 메소드의 입력은 체인의 첫 번째 컴포넌트가 기대하는 형태와 일치해야 합니다.
# 여기서는 lambda 함수의 입력 x, 즉 {"question": user_question} 입니다.
response = rag_chain.invoke({"question": user_question})
print(response)

[사용자 질문]: 포토리소그래피 공정이란 무엇이고, 왜 중요한가요?

[RAG 파이프라인 답변]:
포토리소그래피 공정은 반도체 제조에서 매우 중요한 단계로, 웨이퍼 표면에 미세한 패턴을 형성하는 데 사용됩니다. 이 공정은 빛을 이용하여 감광성 물질인 포토레지스트에 원하는 회로 패턴을 전사하는 과정을 포함합니다. 포토리소그래피는 반도체 소자의 크기와 성능을 결정하는 데 핵심적인 역할을 하며, 미세한 패턴을 정확하게 구현할 수록 더 높은 집적도와 성능을 가진 반도체를 생산할 수 있습니다. 따라서, 이 공정의 정밀도와 정확성은 전체 반도체 제조 공정의 성공에 큰 영향을 미칩니다.


## HNSW 적용해 보기

In [12]:
import hnswlib
import time

# HNSWLib 인덱스 생성 및 저장
print("HNSWLib 인덱스 생성 중...")
dim = embeddings.shape[1]
num_elements = embeddings.shape[0]

index_hnsw = hnswlib.Index(space='l2', dim=dim) # L2(유클리드) 거리 사용
index_hnsw.init_index(max_elements=num_elements, ef_construction=200, M=16)
index_hnsw.add_items(embeddings, np.arange(num_elements))
index_hnsw.set_ef(50)
index_hnsw.save_index("manual_hnsw.index")
print("'manual_hnsw.index' 파일이 저장되었습니다.")

# HNSWLib 기반 검색 함수 구현
def retrieve_from_hnsw(query: str, k: int = 2) -> list[str]:
    """HNSWLib 인덱스에서 쿼리와 가장 관련 높은 k개의 문서를 검색합니다."""
    index = hnswlib.Index(space='l2', dim=dim)
    index.load_index("manual_hnsw.index", max_elements=num_elements)

    query_vector = embed_model.encode([query])
    labels, distances = index.knn_query(query_vector, k=k)

    return [docs[i] for i in labels[0]]

HNSWLib 인덱스 생성 중...
'manual_hnsw.index' 파일이 저장되었습니다.


In [13]:
# --- 성능 비교 실험 ---
print("\n--- 검색 성능 비교 ---")
comparison_query = "CMP 공정은 무엇을 하는 단계인가?"
print(f"[비교 질문]: {comparison_query}\n")

# FAISS 속도 측정
start_time = time.time()
faiss_results = retrieve_from_faiss(comparison_query, k=1)
faiss_time = time.time() - start_time
print(f"FAISS 검색 시간: {faiss_time:.6f}초")
print(f"FAISS 결과: {faiss_results[0][:100]}...")

# HNSWLib 속도 측정
start_time = time.time()
hnsw_results = retrieve_from_hnsw(comparison_query, k=1)
hnsw_time = time.time() - start_time
print(f"\nHNSWLib 검색 시간: {hnsw_time:.6f}초")
print(f"HNSWLib 결과: {hnsw_results[0][:100]}...")

# 데이터가 적을 경우 속도 차이가 미미하거나, 인덱스 로딩 시간 때문에 오히려 FAISS가 빠를 수도 있음


--- 검색 성능 비교 ---
[비교 질문]: CMP 공정은 무엇을 하는 단계인가?

FAISS 검색 시간: 0.009631초
FAISS 결과: ## 1. 서론
이 문서는 최신 반도체 제조 공정에 대한 기술적 세부사항을 다룹니다. 각 공정 단계는 수율과 성능에 직접적인 영향을 미치므로, 모든 파라미터는 엄격하게 관리되어야 ...

HNSWLib 검색 시간: 0.008096초
HNSWLib 결과: ## 1. 서론
이 문서는 최신 반도체 제조 공정에 대한 기술적 세부사항을 다룹니다. 각 공정 단계는 수율과 성능에 직접적인 영향을 미치므로, 모든 파라미터는 엄격하게 관리되어야 ...


In [14]:
# --- HNSWLib를 RAG 파이프라인에 통합하여 최종 답변 생성 ---
print("\n--- HNSWLib 기반 RAG 파이프라인 실행 ---")
rag_chain_hnsw = (
    {"context": (lambda x: "\n---\n".join(retrieve_from_hnsw(x["question"]))), "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
response_hnsw = rag_chain_hnsw.invoke({"question": comparison_query})
print(response_hnsw)


--- HNSWLib 기반 RAG 파이프라인 실행 ---
CMP(Chemical Mechanical Polishing) 공정은 웨이퍼 표면을 화학적 및 기계적 방법을 통해 평탄하게 만드는 단계입니다. 이 과정은 웨이퍼 표면의 단차를 제거하여 다음 공정의 안정성을 확보하는 데 중요한 역할을 합니다. CMP 공정에서 슬러리의 종류와 연마 압력은 주요 변수로 작용합니다.


# 참고 : Query Augment

In [15]:
from langchain.retrievers.multi_query import MultiQueryRetriever

# 하나의 질문을 여러 관점으로 변환
original_query = "반도체 제조 과정에서 웨이퍼 처리 방법은?"

# LLM이 여러 버전의 질문 생성
generated_queries = [
    "웨이퍼 가공 기술에 대해 알려주세요",
    "반도체 웨이퍼의 처리 단계는 무엇인가요?",
    "실리콘 웨이퍼 제조 공정을 설명해주세요" ]

In [16]:
# 원본 질문을 의미적으로 관련된 여러 키워드로 확장
query = "반도체 수율 향상"
expanded_queries = [
    "반도체 수율 향상",
    "웨이퍼 품질 개선",
    "공정 최적화",
    "defect reduction" ]

In [ ]:
# 각 query를 vector로 변환
query_vectors = [embeddings.embed_query(q) for q in generated_queries]

In [ ]:
### 단일 vector로 변환 방법들 ###

# 1) 각각 검색 후 합치기
def multi_vector_search(query_vectors, vectorstore, k=5):
    all_results = []
    # 각 query vector로 개별 검색
    for query_vec in query_vectors:
        results = vectorstore.similarity_search_by_vector(query_vec, k=k)
        all_results.extend(results)
    # 중복 제거 및 점수 합산
    unique_results = remove_duplicates_and_merge_scores(all_results)
    return unique_results[:k]

# 2) Vector 평균화
def average_vector_search(query_vectors, vectorstore, k=5):
    # 여러 query vector의 평균 계산
    avg_vector = np.mean(query_vectors, axis=0)
    # 평균 vector로 검색
    results = vectorstore.similarity_search_by_vector(avg_vector, k=k)
    return results

# 3) 가중 평균
def weighted_vector_search(query_vectors, weights, vectorstore, k=5):
    # 가중 평균 계산 (중요한 query에 더 큰 가중치)
    weighted_avg = np.average(query_vectors, weights=weights, axis=0)

    results = vectorstore.similarity_search_by_vector(weighted_avg, k=k)
    return results

# 4) 최대값 방식 (Max Similarity)
def max_similarity_search(query_vectors, vectorstore, k=5):
    chunk_scores = {}
    # 각 chunk에 대해 모든 query vector와의 유사도 중 최댓값 사용
    for query_vec in query_vectors:
        results_with_scores = vectorstore.similarity_search_with_score_by_vector(query_vec, k=50)
        for chunk, score in results_with_scores:
            chunk_id = chunk.page_content
            if chunk_id not in chunk_scores:
                chunk_scores[chunk_id] = score
            else:
                chunk_scores[chunk_id] = max(chunk_scores[chunk_id], score)
    # 점수 순으로 정렬
    sorted_chunks = sorted(chunk_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_chunks[:k]

# 5) 앙상블 방식
def ensemble_search(query_vectors, vectorstore, k=5):
    method_results = []
    # 여러 방법으로 검색
    method_results.append(average_vector_search(query_vectors, vectorstore, k))
    method_results.append(max_similarity_search(query_vectors, vectorstore, k))
    #  ...
    # 결과 투표/가중 합산
    final_results = combine_results_by_voting(method_results)
    return final_results[:k]


LangChain 구현 예시


In [ ]:
## LangChain 구현 예시

from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOpenAI

# Multi-Query Retriever 설정
llm = ChatOpenAI(temperature=0)
retriever = MultiQueryRetriever.from_llm(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 3}),
    llm=llm
)

# 내부적으로 다음과 같이 동작:
# 1. 원본 질문을 3-5개의 다른 표현으로 변환
# 2. 각각을 vector로 변환하여 검색
# 3. 결과를 합치고 중복 제거
results = retriever.get_relevant_documents("반도체 수율 향상 방법은?")